# Benchmark 2.0 

- This time, we are using a new database : SLKB, which has higher-quality, uniform data
- We are integrating the wilcox displacement estimate into the labeling of our positive results (which should have been done already)
- We will use no negative samples, instead relying on positive vs unlabeled samples
- Positive samples will be benchmarked against interactions found in n different cell lines
- SLKB uses different scoring systems with small overlap. 
  
- We will evaluate the overlap with each scoring system separately, at different thresholds : draw a heatmap table with one column per scoring system, and one row per score threshold, where each square is colored corresponding to the precision of the model.
- This heatmap can be redrawn for different p-value thresholds and numbers of cell lines

In [1]:
import os
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

In [2]:
# This cell allows you to download the necessary SLKB files
# Google drive links come from slkb.osubmi.org

file_urls = {
    # SLKB files, obtrained from slkb.osubmi.org
    "inputs/SLKB_predictions.xlsx": "https://slkb.osubmi.org/session/09c5c4738990db9810156682c87de6b8/download/download_data-predSL?w="
}
# Create the inputs directory if it doesn't exist
os.makedirs("inputs", exist_ok=True)

# Download missing files
for filepath, file_url in file_urls.items():
    if not os.path.exists(filepath):
        print(f"Downloading {filepath}...")
        gdown.download(file_url, filepath, quiet=False)

In [3]:
# This excel file is the output of the model we want to benchmark
# It contains our model's predictions, generated by an R script
model_predictions = pd.read_csv("inputs/Crispr_DepMap_Outputs_7mai.csv")

# We first extract the list of "mutant" genes included in our model, by getting the names of the tabs in the excel file, this will be used later
mutant_genes = model_predictions.keys()
mutant_genes_list = list(mutant_genes)

# Then, we extract a list of "ko" genes, which are simply all of the genes that each mutant (such as ARID1A) was tested against. 
# Of course, this also has to include ARID1A itself.
ko_genes_list = model_predictions[model_predictions['mutant'] == 'ARID1A']['gene'].tolist()
if 'ARID1A' not in ko_genes_list:
    ko_genes_list.append('ARID1A')

print("Length of ko genes list:", len(ko_genes_list))
print("Length of mutant genes list:", len(mutant_genes_list))
print("Rows in model predictions dataframe: ", model_predictions.shape[0])
model_predictions.head()

Length of ko genes list: 10053
Length of mutant genes list: 14
Rows in model predictions dataframe:  1598268


,estimate,p.value,conf.low,conf.high,method,alternative,gene,mutant,high,low,percentile,diff_mean,diff_median,p_adj
0,0.020349,0.653732,-inf,0.107307,Wilcoxon rank sum test with continuity correction,less,EP300,ACTL6A,97,99,NaN,0.005970,0.071251,0.999997
1,-0.000420,0.495981,-inf,0.055589,Wilcoxon rank sum test with continuity correction,less,MICOS10,ACTL6A,97,99,NaN,0.030204,0.007120,0.999997
2,0.022803,0.763557,-inf,0.076529,Wilcoxon rank sum test with continuity correction,less,PDCD10,ACTL6A,97,99,NaN,0.004063,0.039151,0.999997
3,-0.022775,0.299312,-inf,0.046320,Wilcoxon rank sum test with continuity correction,less,SDHD,ACTL6A,97,99,NaN,0.027244,0.027420,0.943160
4,0.037436,0.959758,-inf,0.072021,Wilcoxon rank sum test with continuity correction,less,ROCK1,ACTL6A,97,99,NaN,0.035888,0.036536,0.999997


In [4]:
# This table, from SLKB, shows which SL pairs scored within the top 10% of 5 different scoring systems, among 22 different cell lines.
#
# Downloaded on 04/04/2025 from https://slkb.osubmi.org/,

slkb_predictions = pd.read_excel("inputs/SLKB_predictions.xlsx", sheet_name=None)

# Here, we also have to concatenate the different sheets
slkb_predictions = pd.concat(
  [df.assign(cell_line=name) for name, df in slkb_predictions.items()],
  ignore_index=True
)

# We separate the "gene_pair" column into two separate columns, to match the format of our first table
slkb_predictions[["gene1", "gene2"]] = slkb_predictions["gene_pair"].str.split('|', expand=True)
slkb_predictions.drop(columns="gene_pair", inplace=True)

# According to the SLKB paper, genes are considered "SLi" if they are in the top 10% of at least 3 different scoring systems, so we filter the dataframe accordingly
slkb_predictions_sli = slkb_predictions[slkb_predictions["total_count"] >= 3]

# Group by gene pairs and aggregate cell lines and scoring metrics
slkb_predictions_sli = slkb_predictions_sli.groupby(['gene1', 'gene2']).agg({
    'cell_line': lambda x: ';'.join(sorted(set(x))),
}).reset_index()

print("initial number of SLis: ", slkb_predictions_sli.shape[0])

# We now need to filter the SLi pairs to only include those that are also present in our model predictions
slkb_predictions_sli = slkb_predictions_sli[
    slkb_predictions_sli['gene1'].isin(ko_genes_list) & slkb_predictions_sli['gene2'].isin(ko_genes_list)
]
print("number of SLis after removing non-reactome genes: ", slkb_predictions_sli.shape[0])

# We then filter the SLi pairs to only include those that have been found in at least two different cell lines
slkb_predictions_sli = slkb_predictions_sli[
    slkb_predictions_sli['cell_line'].str.split(';').apply(len) >= 2
]
print("number of SLis after removing row for interactions found in only one cell line: ", slkb_predictions_sli.shape[0])

# Creating a new column to indicate that these are SLi pairs, filled with 1s
slkb_predictions_sli["sli"] = 1

slkb_predictions_sli

initial number of SLis:  10455
number of SLis after removing non-reactome genes:  5464
number of SLis after removing row for interactions found in only one cell line:  791


,gene1,gene2,cell_line,sli
5,AARS2,PTTG1,30033366_JURKAT;30033366_K562,1
29,ABCB7,HSCB,30033366_JURKAT;30033366_K562,1
37,ABCB7,OPA1,30033366_JURKAT;30033366_K562,1
39,ABCB7,PITRM1,30033366_JURKAT;30033366_K562,1
61,ACO1,IREB2,34469736_HELA;34469736_PC9,1
...,...,...,...,...
10379,TUBGCP2,TUBGCP3,33637726_A375;33637726_MEWO,1
10396,UBE2A,UBE2B,34469736_PC9;34857952_A549;34857952_GI1;348579...,1
10401,UPF3A,UPF3B,33637726_A375;33637726_MEWO,1
10410,USP15,USP4,34857952_GI1;34857952_MEL202,1


In [5]:
# Now we export the number of "unique" mutants to feed our R model so that our screening is more complete
unique_mutants_sli = slkb_predictions_sli["gene1"].unique()

print("number of unique mutants: ", len(unique_mutants_sli))

# Saving it to a csv
np.savetxt("outputs/unique_mutants_sli_slkb.csv", unique_mutants_sli, delimiter=",", fmt="%s")

number of unique mutants:  330


# Filtering the dataset to only keep intersecting data points

In [6]:
# First, we get the sets of positive pairs in the benchmark dataset
# Negative pairs can be optionally included, depending on whether this is set to True or False
include_non_sli_data = False

# Create sets of gene pairs from the benchmark for efficient lookup
benchmark_sli_set =  set()
for index, row in slkb_predictions_sli[slkb_predictions_sli["sli"] == 1].iterrows():
    gene1 = row['gene1']
    gene2 = row['gene2']
    benchmark_pair_sorted = tuple(sorted((gene1, gene2)))
    benchmark_sli_set.add(benchmark_pair_sorted)

print("Number of SLIs in the benchmark set: ", len(benchmark_sli_set))

# If include_non_sli_data is set to false, the set will be empty, and we won't include the non-sli data in the benchmark
if include_non_sli_data:
    benchmark_nonsli_set = set()
    for index, row in synlethdb_predictions[synlethdb_predictions["sli"] == 0].iterrows():
        gene1 = row['gene1']
        gene2 = row['gene2']
        benchmark_pair_sorted = tuple(sorted((gene1, gene2)))
        benchmark_nonsli_set.add(benchmark_pair_sorted)

print("Number of non-SLIs in the benchmark set: ", len(benchmark_nonsli_set)) if include_non_sli_data else print("Non-SLIs are currently not included.")

Number of SLIs in the benchmark set:  791
Non-SLIs are currently not included.


In [7]:
# Then, we filter the huge model predictions dataframe to keep only the rows that are in the benchmark dataset
filtered_rows = []
model_pairs_set = set()

for index, row in model_predictions.iterrows():
    mutant = row['mutant']
    gene = row['gene']
    pair_sorted = tuple(sorted((mutant, gene)))

    if pair_sorted in benchmark_sli_set:
        filtered_rows.append(row)
        model_pairs_set.add(pair_sorted)

    elif include_non_sli_data and pair_sorted in benchmark_nonsli_set:
        filtered_rows.append(row)
        model_pairs_set.add(pair_sorted)

model_predictions_filtered = pd.DataFrame(filtered_rows)

print("Shape of the filtered model predictions:", model_predictions_filtered.shape)

# Note that there are a number of duplicate rows with different values. We keep the highest value for each pair only 
model_predictions_filtered['sorted_pair'] = model_predictions_filtered.apply(lambda row: tuple(sorted((row['mutant'], row['gene']))), axis=1)
idx = model_predictions_filtered.groupby('sorted_pair')['p_adj'].idxmin()
model_predictions_filtered = model_predictions_filtered.loc[idx]
model_predictions_filtered = model_predictions_filtered.drop(columns=['sorted_pair'])

print("Shape of model predictions without duplicate gene pairs :", model_predictions_filtered.shape)

Shape of the filtered model predictions: (343, 14)
Shape of model predictions without duplicate gene pairs : (251, 14)


In [8]:
# Finally, we remove pairs in our benchmark dataset that are not representer in our model's dataset
benchmark_sli_set_filtered = set()
benchmark_nonsli_set_filtered = set()

for pair in benchmark_sli_set:
    if pair in model_pairs_set:
        benchmark_sli_set_filtered.add(pair)
        benchmark_sli_set = benchmark_sli_set_filtered
print("Number of SLis in benchmark set after excluding pairs not in the model dataset: ", len(benchmark_sli_set))

if include_non_sli_data:
    for pair in benchmark_nonsli_set:
        if pair in model_pairs_set:
            benchmark_nonsli_set_filtered.add(pair)
    benchmark_nonsli_set = benchmark_nonsli_set_filtered
    print("Number of Non-SLis in benchmark set after excluding pairs not in the model dataset: ", len(benchmark_nonsli_set))


Number of SLis in benchmark set after excluding pairs not in the model dataset:  251


### Positive classification conditions for our model : 
- Wilcox displacement estimate over 0.15
- p-value under 1e-3 or 1e-4 (to be adjusted)
- for the gene pairs that were tested multiple times, one positive classification is sufficient

### SLKB conditions for positive classification
- It depends on the scoring system
- It depends on the number of cell lines in which it's been identified (2+)

In [9]:
# Create a set of unique pairs from the benchmark database, based on a p_adj threshold
def get_model_classifications():
    """Preprocess the data once for all thresholds"""
    # Create a mapping from gene pairs to their p_adj values
    pair_to_p_adj = {}
    
    for _, row in model_predictions_filtered.iterrows():
        gene = row['gene']
        mutant = row['mutant']
        model_pair_sorted = tuple(sorted((gene, mutant)))
        
        # Only consider pairs in our benchmark sets
        if model_pair_sorted in benchmark_sli_set or model_pair_sorted in benchmark_nonsli_set:
            # Take the lowest p_adj if multiple entries exist
            if model_pair_sorted not in pair_to_p_adj or row['p_adj'] < pair_to_p_adj[model_pair_sorted]:
                pair_to_p_adj[model_pair_sorted] = row['p_adj']
    
    return pair_to_p_adj

pair_to_p_adj = get_model_classifications()

In [10]:
def get_confusion_sets(p_adj_threshold, pair_to_p_adj, print_tables=False):
    """Fast version using preprocessed data"""
    # Generate model_sli_set and model_nonsli_set for this threshold
    model_sli_set = {pair for pair, p_adj in pair_to_p_adj.items() if p_adj <= p_adj_threshold}
    model_nonsli_set = {pair for pair, p_adj in pair_to_p_adj.items() if p_adj > p_adj_threshold}
    
    # Calculate confusion matrix values
    confusion = {
        "TP": model_sli_set.intersection(benchmark_sli_set),
        "FP": model_sli_set.intersection(benchmark_nonsli_set) if include_non_sli_data else set(),
        "TN": model_nonsli_set.intersection(benchmark_nonsli_set) if include_non_sli_data else set(),
        "FN": model_nonsli_set.intersection(benchmark_sli_set)
    }
    
    if print_tables:
        print(f"True Positives (TP): {len(confusion['TP'])}")
        for item in confusion["TP"]:
            print(f"{item}: {pair_to_p_adj[item]}")
        print(f"False Positives (FP): {len(confusion['FP'])}")
        print(f"True Negatives (TN): {len(confusion['TN'])}")
        print(f"False Negatives (FN): {len(confusion['FN'])}")
        
    return confusion
    

def get_recall(confusion):
    TP = len(confusion["TP"])
    FN = len(confusion["FN"])
    recall = TP / (TP + FN) if (TP + FN) > 0 else None
    return recall


def get_precision(confusion):
    TP = len(confusion["TP"])
    FP = len(confusion["FP"])
    precision = TP / (TP + FP) if (TP + FP) > 0 else None
    return precision

# This metric is similar to the F1 score but for Positive/Unlabeled (PU) Data 
# the positive_prediction_ratio corresponds to Pr(ŷ = 1), so the proportion of total model predictions that are positive
def get_pu_metric(confusion):
    recall = get_recall(confusion)
    positive_prediction_ratio = len(confusion["TP"]) / (len(confusion["TP"]) + len(confusion["FP"]))
    pu_metric = (recall * recall) / positive_prediction_ratio if positive_prediction_ratio > 0 else None
    return pu_metric

p_adj_treshold=0.001

confusion = get_confusion_sets(p_adj_treshold, pair_to_p_adj, print_tables=True)
precision = get_precision(confusion)
recall = get_recall(confusion)
# pu_metric = get_pu_metric(confusion)

print("true positives: ", len(confusion["TP"]))
print("false positives: ", len(confusion["FP"])) if include_non_sli_data else print("")
print("true negatives: ", len(confusion["TN"])) if include_non_sli_data else print("")
print("false negatives: ", len(confusion["FN"]))
print("Precision: ",precision) if include_non_sli_data else print("")
print("Recall: ", recall)
# print("PU metric: ", pu_metric) if not include_non_sli_data else print("")

True Positives (TP): 4
('EFR3A', 'LEO1'): 3.9283519054352725e-05
('SMARCA2', 'SMARCA4'): 2.491069321917753e-09
('ARL2', 'PPP2R1A'): 6.0502828901674944e-05
('GTF3C3', 'TUBB'): 9.808810448653378e-06
False Positives (FP): 0
True Negatives (TN): 0
False Negatives (FN): 247
true positives:  4


false negatives:  247

Recall:  0.01593625498007968


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the p-value thresholds
p_adj_thresholds = np.linspace(0.0000000001, 0.0001, 10000)

plt.figure(figsize=(8, 6))

precisions = []
recalls = []

for thresh in p_adj_thresholds:
    confusion = get_confusion_sets(thresh, pair_to_p_adj)
    precision = get_precision(confusion)
    recall = get_recall(confusion)
    
    if precision is not None and recall is not None:
        precisions.append(precision)
        recalls.append(recall)

plt.plot(recalls, precisions, marker='o')

# Final plot adjustments
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title(f"Precision-Recall Curve")
plt.grid(True)
plt.tight_layout()
plt.show()


In [7]:
# Now, we want to generate a heatmap for different scoring systems and cell line numbers
def draw_precision_recall_table(p_adj_treshold, cell_line_numbers=[1, 2, 3, 4, 5], total_count=None):
    hit_rates_dict = {}
    hit_rates_dict["num_cell_lines"] = cell_line_numbers

    for scoring_system in scoring_systems_list:
        if scoring_system == "consensus" and total_count == None:
            continue
        # Create a column containing the precision for all cell line numbers
        column_name = scoring_system + "_Precision" if scoring_system != "consensus" else scoring_system + "_>=" + str(total_count) + "_Precision"
        hit_rates_dict[column_name] = []
        for num_cell_lines in cell_line_numbers:
            hit_rates_dict[column_name].append(get_precision(p_adj_treshold, scoring_system, num_cell_lines, total_count))

        # Create a column containing the recall for all cell line numbers
        column_name = scoring_system + "_Recall" if scoring_system != "consensus" else scoring_system + "_>=" + str(total_count) + "_Recall"
        hit_rates_dict[column_name] = []
        for num_cell_lines in cell_line_numbers:
            hit_rates_dict[column_name].append(get_recall(p_adj_treshold, scoring_system, num_cell_lines, total_count))

    hit_rates_df = pd.DataFrame(hit_rates_dict)
    return hit_rates_df

p_adj_treshold = 0.001
hit_rates_df = draw_precision_recall_table(p_adj_treshold, total_count=3)

hit_rates_df

,num_cell_lines,GEMINI-Score_Precision,GEMINI-Score_Recall,HORLBECK-Score_Precision,HORLBECK-Score_Recall,MAGECK-Score_Precision,MAGECK-Score_Recall,MEDIAN-B/NB Score_Precision,MEDIAN-B/NB Score_Recall,sgRNA-Derived B/NB Score_Precision,sgRNA-Derived B/NB Score_Recall,consensus_>=3_Precision,consensus_>=3_Recall
0,1,0.008696,0.016529,0.008696,0.011561,0.004348,0.008065,0.004348,0.008696,0.008696,0.01626,0.004348,0.016393
1,2,0.004348,0.052632,0.004348,0.052632,0.004348,0.111111,0.000000,0.000000,0.004348,0.06250,0.004348,0.333333
2,3,0.004348,0.200000,0.004348,0.500000,0.004348,0.500000,0.000000,0.000000,0.004348,0.50000,0.004348,1.000000
3,4,0.000000,0.000000,0.004348,1.000000,0.004348,1.000000,0.000000,NaN,0.004348,1.00000,0.000000,NaN
4,5,0.000000,0.000000,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN


In [ ]:
p_adj_treshold = 0.001
scoring_system = "consensus"
num_cell_lines = 3
test = get_confusion_sets(p_adj_treshold, scoring_system, num_cell_lines, total_count=3, print_tables=True)
precision = get_precision(p_adj_treshold, scoring_system, num_cell_lines, total_count=3)
recall = get_recall(p_adj_treshold, scoring_system, num_cell_lines, total_count=3)
print(f"precision: {precision}")
print(f"recall: {recall}")

True Positives (TP): 1
{('SMARCA2', 'SMARCA4')}
False Positives (FP): 229
{('PBRM1', 'UBA5'), ('EED', 'TAF1D'), ('COQ4', 'KMT2C'), ('ATP6V1G1', 'PBRM1'), ('BAP1', 'FAM50A'), ('PBRM1', 'SEC61A1'), ('ARID2', 'SDHD'), ('COX5B', 'SMARCA4'), ('ARID1A', 'MRPL36'), ('KLF5', 'PBRM1'), ('EFR3A', 'PBRM1'), ('ARID1B', 'SEPSECS'), ('MICOS10', 'SETD2'), ('ARID1B', 'TAF10'), ('ARID1A', 'ATP1B3'), ('KMT2C', 'UQCRC1'), ('ARID1A', 'MICOS10'), ('BAP1', 'CDS2'), ('COPS4', 'PBRM1'), ('PBRM1', 'VPS37A'), ('SEPHS2', 'SMARCA4'), ('ARHGEF7', 'PBRM1'), ('ARID2', 'COX5B'), ('SMARCA2', 'TRIM37'), ('CRKL', 'SMARCA2'), ('ARID1B', 'CDS2'), ('ARID2', 'RARS2'), ('LIMS1', 'PBRM1'), ('KMT2C', 'SDHD'), ('SDHD', 'SMARCA4'), ('FECH', 'SMARCA4'), ('SMARCA4', 'TTF2'), ('ARID2', 'TOMM22'), ('EED', 'WWTR1'), ('EP300', 'SETD2'), ('KMT2C', 'SUPT7L'), ('NAMPT', 'SMARCA4'), ('EP300', 'KMT2D'), ('ADSL', 'KMT2C'), ('BAP1', 'SEPHS2'), ('EP300', 'KMT2C'), ('PBRM1', 'UBA52'), ('KMT2C', 'MRPS21'), ('BAP1', 'HSPA8'), ('MARK2', 'PBRM1'),

It seems that the only gene pair on SLKB that was labeled in 4 or more cell lines by any score was SMARCA2/SMARCA4, which incidentally happens to be labeled by the model. Let's see what other pairs are on SLKB if we remove the filter for "mutant" genes : it may give us additional pairs to try

In [9]:
# With the same code from the beginning, we generate a 2nd table where we filter differently to keep all gene pairs that contain 2 "ko" genes instead of "mutants"

# This table, from SLKB, shows which SL pairs scored within the top 10% of 5 different scoring systems, among 22 different cell lines.
#
# Downloaded on 04/04/2025 from https://slkb.osubmi.org/, specifically here https://slkb.osubmi.org/session/00b5c112434073acd4ea9e9668422bec/download/download_data-predSL?w= 

slkb_predictions2 = pd.read_excel("inputs/SLKB_predictions.xlsx", sheet_name=None)

# Here, we also have to concatenate the different sheets
slkb_predictions2 = pd.concat(
  [df.assign(cell_line=name) for name, df in slkb_predictions2.items()],
  ignore_index=True
)

# We separate the "gene_pair" column into two separate columns, to match the format of our first table
slkb_predictions2[["gene1", "gene2"]] = slkb_predictions2["gene_pair"].str.split('|', expand=True)
slkb_predictions2.drop(columns="gene_pair", inplace=True)

# Finally, we filter it to only include the gene pairs where one of the studied genes is a "mutant" from our dataset, and the other is a ko gene
slkb_predictions2 = slkb_predictions2[
    ((slkb_predictions2["gene1"].isin(ko_genes_list)) & slkb_predictions2["gene2"].isin(ko_genes_list))
]

slkb_predictions2.head()

,GEMINI-Score,HORLBECK-Score,MAGECK-Score,MEDIAN-B/NB Score,sgRNA-Derived B/NB Score,total_count,cell_line,gene1,gene2
0,1,1,1,1,1,5,26864203_OVCAR8,KDM1B,MBD1
1,1,1,1,1,1,5,26864203_OVCAR8,KDM4A,KDM5A
2,1,1,1,1,1,5,26864203_OVCAR8,KDM4C,KDM5A
3,1,1,1,0,1,4,26864203_OVCAR8,BMI1,KDM1B
4,0,1,1,1,1,4,26864203_OVCAR8,CREBBP,ING5


In [10]:
def get_positives_list_enlarged(p_adj_treshold, scoring_system, num_cell_lines, print_tables=False):
    # Get the list of "hit pairs" at a given adjusted p-value threshold, where the estimate is under -0.15 (since the sign was inverted by the model)
    hits_data = model_predictions[(model_predictions["estimate"] < -0.15) & (model_predictions["p_adj"] < p_adj_treshold)].copy()

    # Normalize hits_data
    hits_data[["mutant", "gene"]] = hits_data[["mutant", "gene"]].apply(lambda row: sorted(row), axis=1, result_type="expand")
    hits_data = hits_data.rename(columns={"mutant": "gene1", "gene": "gene2"})
    hits_data_unique = hits_data[["gene1", "gene2"]].drop_duplicates()

    # Find positive results
    positive_results = slkb_predictions2[slkb_predictions2[scoring_system] == 1]

    # Normalize gene pairs
    positive_results_normalized = positive_results.copy()
    positive_results_normalized[["gene1", "gene2"]] = (
        positive_results_normalized[["gene1", "gene2"]]
        .apply(lambda row: sorted(row), axis=1, result_type="expand")
    )

    # Group by the normalized pairs and add the number of positive results
    positive_results_count = positive_results_normalized.groupby(["gene1", "gene2"]).sum().reset_index()

    # Filter to keep only the results that have been found in num_cell_lines cell lines
    positive_benchmark_df = positive_results_count[positive_results_count[scoring_system] >= num_cell_lines]

    # Convert both to sets of tuples
    predicted_pairs = set(map(tuple, hits_data_unique[["gene1", "gene2"]].values))
    known_pairs = set(map(tuple, positive_benchmark_df[["gene1", "gene2"]].values))
    true_positives = predicted_pairs & known_pairs

    if print_tables:
        print("true positives :  \n", true_positives, "\n\n known pairs:  \n", known_pairs, "\n\n predicted pairs:  \n", predicted_pairs) 

    return known_pairs, true_positives

It looks like out of 268 different pairs detected in at least 4 cell lines, only 4 or 5 are detectable by the algorithm (tested in a separate notebook).

- revoir étapes de filtrage sur synlethdb
- lancer algo pour ~200 genes synlethdb et slkb
- rediger mail avec questions pr jorge